# Preparation

In [70]:
pip install gensim --user

Note: you may need to restart the kernel to use updated packages.


In [71]:
#Import

import gensim
import pandas as pd
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [72]:
# Gensim import dan load pre-trained idwiki language model

path = '../id_wiki/idwiki_word2vec_300.model'
id_w2v = gensim.models.word2vec.Word2Vec.load(path)
print(id_w2v.wv.most_similar('tes'))

[('pengujian', 0.7212876677513123), ('uji', 0.7056494951248169), ('pemeriksaan', 0.6564321517944336), ('ujian', 0.5918323993682861), ('percobaan', 0.5823646187782288), ('eksperimen', 0.5750775933265686), ('test', 0.550246000289917), ('serologis', 0.5315841436386108), ('verifikasi', 0.5280280113220215), ('ujicoba', 0.5256417393684387)]


# Prepare Dataset

In [73]:
#Dataset Preparation

yelp = pd.read_csv('dataset.csv', sep=',', header = 0)
del yelp['No']
yelp = yelp.dropna()
yelp.head()

,jenis,uraian,pertanyaan,opsi1,opsi2,opsi3,opsi4,kunci_jawaban,kategori
0,Makna kata/istilah,Padi yang luas menguning adalah hasil tanam ya...,Makna kata jasa pada paragraf di atas adalah,Pelayanan yang terbaik bagi kita.\t,Manfaat yang melimpah bagi kita\t,Perbuatan yang berguna bagi orang lain\t,Jerih payah yang sangat menguntungkan,Perbuatan yang berguna bagi orang lain\t,Mudah
1,Antonim/sinonim,Suara radio tetangga sebelah sangat keras pada...,Antonim kata keras pada kalimat tersebut adalah,pelan\t,lunak\t,kaku\t,kencang,pelan\t,Mudah
2,Informasi tersurat teks,Pembiasaan Hidup Sehat Sejak KecilPola hidup s...,Bagaimana cara memenuhi gizi anak pada usia ba...,Menambah gizi anak seiring bertambahnya usia a...,"Makan makanan bergizi, istirahat cukup, dan ol...",Mempunyai kekebalan yang baik terhadap seranga...,Pemenuhan gizi dengan pemberian ASI saat anak...,Pemenuhan gizi dengan pemberian ASI saat anak...,Sedang
3,Informasi tersirat teks,Mulai tahun 2016 kuota haji di Indonesssia dit...,Berdasarkan data kuota jumlah jamaah haji untu...,77,77.7,78,"77,800",77,Sedang
4,Informasi tersurat teks,Kebiasaan makan makanan bergizi dengan kadar s...,Kalimat dalam paragraf di atas yang tidak padu...,Kebutuhan gizi anak akan semakin bertambah sei...,Kebutuhan gizi anak baru terpenuhi saat anak b...,Tubuh anak mudah terserang berbagai macam peny...,Pertumbuhan dan perkembangan anak akan semakin...,Tubuh anak mudah terserang berbagai macam peny...,Sedang


In [74]:
#Data Cleaning
yelp = yelp.replace('\n',' ', regex=True)
yelp = yelp.replace('\t',' ', regex=True)
yelp = yelp.replace('"',' " ', regex=True)
yelp.head()

,jenis,uraian,pertanyaan,opsi1,opsi2,opsi3,opsi4,kunci_jawaban,kategori
0,Makna kata/istilah,Padi yang luas menguning adalah hasil tanam ya...,Makna kata jasa pada paragraf di atas adalah,Pelayanan yang terbaik bagi kita.,Manfaat yang melimpah bagi kita,Perbuatan yang berguna bagi orang lain,Jerih payah yang sangat menguntungkan,Perbuatan yang berguna bagi orang lain,Mudah
1,Antonim/sinonim,Suara radio tetangga sebelah sangat keras pada...,Antonim kata keras pada kalimat tersebut adalah,pelan,lunak,kaku,kencang,pelan,Mudah
2,Informasi tersurat teks,Pembiasaan Hidup Sehat Sejak KecilPola hidup s...,Bagaimana cara memenuhi gizi anak pada usia ba...,Menambah gizi anak seiring bertambahnya usia a...,"Makan makanan bergizi, istirahat cukup, dan ol...",Mempunyai kekebalan yang baik terhadap seranga...,Pemenuhan gizi dengan pemberian ASI saat anak...,Pemenuhan gizi dengan pemberian ASI saat anak...,Sedang
3,Informasi tersirat teks,Mulai tahun 2016 kuota haji di Indonesssia dit...,Berdasarkan data kuota jumlah jamaah haji untu...,77,77.7,78,"77,800",77,Sedang
4,Informasi tersurat teks,Kebiasaan makan makanan bergizi dengan kadar s...,Kalimat dalam paragraf di atas yang tidak padu...,Kebutuhan gizi anak akan semakin bertambah sei...,Kebutuhan gizi anak baru terpenuhi saat anak b...,Tubuh anak mudah terserang berbagai macam peny...,Pertumbuhan dan perkembangan anak akan semakin...,Tubuh anak mudah terserang berbagai macam peny...,Sedang


In [75]:
df = pd.DataFrame()

#Kolom teks merupakan gabungan dari kolom jenis soal, uraian soal, pertanyaan soal, opsi1 soal, opsi2 soal, opsi3 soal, opsi4 soal, dan kunci jawaban soal
df['teks'] = yelp['jenis'].map(str) + ' ' + yelp['uraian'].map(str)+ ' ' + yelp['pertanyaan'].map(str)+ ' ' + yelp['opsi1'].map(str)+ ' ' + yelp['opsi2'].map(str)+ ' ' + yelp['opsi3'].map(str)+ ' ' + yelp['opsi4'].map(str)+ ' ' + yelp['kunci_jawaban'].map(str)

#kolom kategori merupakan kolom kategori kesulitan soal
df['kategori'] = yelp['kategori']

yelp = yelp.dropna()
print(df)

                                                  teks kategori
0    Makna kata/istilah Padi yang luas menguning ad...    Mudah
1    Antonim/sinonim Suara radio tetangga sebelah s...    Mudah
2    Informasi tersurat teks Pembiasaan Hidup Sehat...   Sedang
3    Informasi tersirat teks Mulai tahun 2016 kuota...   Sedang
4    Informasi tersurat teks Kebiasaan makan makana...   Sedang
..                                                 ...      ...
413  Tanda baca Merdeka, merdeka!  " Teruskan geril...    Mudah
414  Kata bentukan Tebing Breksi adalah salah satu ...    Sulit
415  Tanda baca  " Sebaiknya kita membawa buah tang...    Mudah
416  Istilah/kata Permasalahan yang dihadapi Nia se...    Sulit
417  Tanda baca Joya berkata,  " Wow, Alangkah inda...    Mudah

[418 rows x 2 columns]


# Data Pre-processing

In [76]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fikri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Stopwords & Embedding Vectorization(Word2Vec)

In [77]:
#Pembentukan dataframe untuk vektorisasi embedding

docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('indonesian') # removing stop words
for doc in yelp['pertanyaan'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = id_w2v.wv[word] # if word is present in embeddings(id_wikidata(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
docs_vectors.shape

C:\Users\fikri\AppData\Local\Temp/ipykernel_13632/1651433560.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  for doc in yelp['pertanyaan'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it


(418, 300)

In [78]:
#Pengecekan terhadap null values pada dataframe docs_vector

pd.isnull(docs_vectors).sum().sum()

0

In [79]:
#transformasi kolom kategori yang semula teks menuju bentuk angka

df['kategori'] = df['kategori'].astype("category")
docs_vectors['label'] = df['kategori'].cat.codes
docs_vectors = docs_vectors.dropna()

In [80]:
docs_vectors

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,label
0,0.093812,0.910717,0.748680,-0.212396,-0.443239,-0.154453,-0.628092,-0.824973,1.061265,0.680660,...,0.623285,-0.041852,0.437371,-0.425462,-0.231662,-0.238558,-1.047369,-1.162888,0.332752,0
1,-0.499552,0.311202,0.612042,0.199530,-0.295520,-0.742239,0.466372,0.323293,1.614589,0.538872,...,0.784201,0.327922,0.226484,-0.971885,-0.912935,-0.815786,-0.203898,0.136879,-0.819996,0
2,0.387774,0.006310,-1.225229,-0.609539,-0.389984,-0.793387,0.978349,0.241335,-0.319050,1.592940,...,1.148679,0.912032,-0.872381,-0.625829,0.936588,-0.215699,0.813238,-0.039967,0.995719,1
3,0.455862,-0.089912,0.193782,-0.180915,0.513921,-0.523443,0.261975,-0.253474,0.045424,0.371530,...,0.898686,0.620154,-0.940243,-1.027972,0.107930,1.226029,1.689125,-0.083854,1.090524,1
4,-0.027289,1.474421,0.946630,0.845040,-0.100184,-0.130372,-0.218282,-0.800743,1.121727,0.689062,...,0.715108,0.271125,0.378930,-0.001044,-1.441504,-0.945212,-0.617464,-0.428226,-0.157788,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.436686,1.855485,0.937091,0.127998,-0.441881,0.261839,-0.501716,0.013089,1.640433,0.026372,...,1.011819,-0.532413,0.617611,-0.208260,-0.781996,-0.921295,0.647014,-0.481330,-0.031824,0
414,0.108022,1.346645,1.258617,1.369551,-0.238920,0.102679,-0.658249,-0.130409,1.007510,0.238095,...,-0.005859,-0.042787,0.044368,0.477196,-1.058346,-0.976184,0.138507,-0.343125,-0.343410,2
415,0.750866,1.638259,1.145283,1.039541,-0.522702,-0.265096,-0.300420,-0.276061,1.482316,0.170811,...,0.796212,-0.793241,0.687771,-0.162560,-0.792651,-0.782765,0.298970,-0.238115,-0.144908,0
416,-0.366664,0.981922,0.931789,0.151190,0.002067,0.315443,-1.006627,0.047238,0.590573,-0.067386,...,0.639390,0.146421,-0.080176,0.072146,0.106544,-0.843861,0.581405,-0.400826,0.610754,2


## Jaccard Similarity

In [81]:
#JACCARD FUNCTION

#define Jaccard Similarity function
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [82]:
#Function Perhitungan Jaccard untuk 2 Kolom
def jacc_calc(col1,col2) :
    res_temp = []
    
    for x, y in zip(col1, col2):
        # x is from a, y is from b
        res = jaccard(x,y)
        res_temp.append(res)
        
    return res_temp    

In [83]:
#Kalkulasi jaccard similarity antara setiap uraian dengan setiap opsi1 hingga opsi4
docs_vectors['jacc_opsi1'] = jacc_calc(yelp['uraian'],yelp['opsi1'])
docs_vectors['jacc_opsi2'] = jacc_calc(yelp['uraian'],yelp['opsi2'])
docs_vectors['jacc_opsi3'] = jacc_calc(yelp['uraian'],yelp['opsi3'])
docs_vectors['jacc_opsi4'] = jacc_calc(yelp['uraian'],yelp['opsi4'])

In [84]:
docs_vectors

,0,1,2,3,4,5,6,7,8,9,...,295,296,297,298,299,label,jacc_opsi1,jacc_opsi2,jacc_opsi3,jacc_opsi4
0,0.093812,0.910717,0.748680,-0.212396,-0.443239,-0.154453,-0.628092,-0.824973,1.061265,0.680660,...,-0.231662,-0.238558,-1.047369,-1.162888,0.332752,0,0.050179,0.050542,0.049296,0.057143
1,-0.499552,0.311202,0.612042,0.199530,-0.295520,-0.742239,0.466372,0.323293,1.614589,0.538872,...,-0.912935,-0.815786,-0.203898,0.136879,-0.819996,0,0.103448,0.103448,0.067797,0.083333
2,0.387774,0.006310,-1.225229,-0.609539,-0.389984,-0.793387,0.978349,0.241335,-0.319050,1.592940,...,0.936588,-0.215699,0.813238,-0.039967,0.995719,1,0.021767,0.028025,0.023929,0.027778
3,0.455862,-0.089912,0.193782,-0.180915,0.513921,-0.523443,0.261975,-0.253474,0.045424,0.371530,...,0.107930,1.226029,1.689125,-0.083854,1.090524,1,0.002320,0.004630,0.004651,0.009259
4,-0.027289,1.474421,0.946630,0.845040,-0.100184,-0.130372,-0.218282,-0.800743,1.121727,0.689062,...,-1.441504,-0.945212,-0.617464,-0.428226,-0.157788,1,0.031746,0.034545,0.037106,0.031423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.436686,1.855485,0.937091,0.127998,-0.441881,0.261839,-0.501716,0.013089,1.640433,0.026372,...,-0.781996,-0.921295,0.647014,-0.481330,-0.031824,0,0.046154,0.046154,0.031746,0.040000
414,0.108022,1.346645,1.258617,1.369551,-0.238920,0.102679,-0.658249,-0.130409,1.007510,0.238095,...,-1.058346,-0.976184,0.138507,-0.343125,-0.343410,2,0.077778,0.066667,0.068182,0.056818
415,0.750866,1.638259,1.145283,1.039541,-0.522702,-0.265096,-0.300420,-0.276061,1.482316,0.170811,...,-0.792651,-0.782765,0.298970,-0.238115,-0.144908,0,0.042169,0.042169,0.042169,0.035928
416,-0.366664,0.981922,0.931789,0.151190,0.002067,0.315443,-1.006627,0.047238,0.590573,-0.067386,...,0.106544,-0.843861,0.581405,-0.400826,0.610754,2,0.049140,0.051724,0.049383,0.051852


# Generate CSV Dataset for Evaluation

In [85]:
docs_vectors.to_csv('dataset_eval_w2vjacc.csv', index=False, encoding='utf-8')

print("Dataset for evauation successfully generated !")# Generate CSV Dataset for Evaluation

Dataset for evauation successfully generated !


# Train & Test Dataset Split

In [86]:
#Persiapan evaluasi - split dataset dan pemilihan teknik classifier

df_eval = dataset = pd.read_csv('dataset_eval_w2vjacc.csv', header=0, engine='python')

train_x, test_x, train_y, test_y = train_test_split(df_eval.drop('label', axis = 1),
                                                   df_eval['label'],
                                                   test_size = 0.15, random_state=42)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((355, 304), (355,), (63, 304), (63,))

# Single Fold Evaluation

In [87]:
#Evaluasi akurasi klasifikasi

model = RandomForestClassifier(n_estimators=300)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_pred)

0.6984126984126984

In [88]:
#predicted y
y_pred = model.predict(test_x)

In [89]:
from sklearn import metrics

print(metrics.classification_report(test_y, y_pred, target_names=['Mudah', 'Sedang', 'Sulit']))

              precision    recall  f1-score   support

       Mudah       0.79      0.92      0.85        24
      Sedang       0.45      0.38      0.42        13
       Sulit       0.71      0.65      0.68        26

    accuracy                           0.70        63
   macro avg       0.65      0.65      0.65        63
weighted avg       0.69      0.70      0.69        63



# Machine Learning Classification Evaluation

## Random Forest Classifier 

In [90]:
from sklearn.ensemble import RandomForestClassifier

acc_temp = 0
prec_temp = 0
rec_temp = 0
f1_temp = 0
n_fold = 10

for i in range(n_fold):
    model = RandomForestClassifier(n_estimators = 300)
    model.fit(train_x, train_y)

    test_pred = model.predict(test_x)
    from sklearn.metrics import accuracy_score
    temp_res = accuracy_score(test_y, test_pred)
    acc_temp+= temp_res
    
    precision, recall, f_value, support = precision_recall_fscore_support(test_y, test_pred, average='macro')
    
    prec_temp+=precision
    rec_temp += recall
    f1_temp += f_value
    
    print(f'Fold {i+1} Eval...   | ACC : {temp_res} | PREC : {precision} | REC : {recall} | F1-SCORE : {f_value} | SUPP : {support}')

res = acc_temp/n_fold
prec_res = prec_temp/n_fold
rec_res = rec_temp/n_fold
f1_res = f1_temp/n_fold

print(f'Average Accuracy Score({n_fold} Times Evaluation) : {res}')
print(f'Average Precision Score({n_fold} Times Evaluation) : {prec_res}')
print(f'Average Recall Score({n_fold} Times Evaluation) : {rec_res}')
print(f'Average F1 Score({n_fold} Times Evaluation) : {f1_res}')

Fold 1 Eval...   | ACC : 0.7142857142857143 | PREC : 0.6609401709401709 | REC : 0.6645299145299145 | F1-SCORE : 0.6619607843137255 | SUPP : None
Fold 2 Eval...   | ACC : 0.7142857142857143 | PREC : 0.6690408357075025 | REC : 0.6645299145299145 | F1-SCORE : 0.6632784663475201 | SUPP : None
Fold 3 Eval...   | ACC : 0.746031746031746 | PREC : 0.7037037037037036 | REC : 0.6901709401709403 | F1-SCORE : 0.6906691779055819 | SUPP : None
Fold 4 Eval...   | ACC : 0.7301587301587301 | PREC : 0.6728395061728395 | REC : 0.6784188034188036 | F1-SCORE : 0.6739869281045752 | SUPP : None
Fold 5 Eval...   | ACC : 0.6984126984126984 | PREC : 0.6466049382716049 | REC : 0.6517094017094017 | F1-SCORE : 0.6475816993464053 | SUPP : None
Fold 6 Eval...   | ACC : 0.6984126984126984 | PREC : 0.659482224699616 | REC : 0.6634615384615384 | F1-SCORE : 0.6596484747745252 | SUPP : None
Fold 7 Eval...   | ACC : 0.6666666666666666 | PREC : 0.6172013708245593 | REC : 0.6260683760683761 | F1-SCORE : 0.6198684602045946 |

## Naive Bayes Classifier 

In [91]:
from sklearn.naive_bayes import GaussianNB

acc_temp = 0
prec_temp = 0
rec_temp = 0
f1_temp = 0
n_fold = 10

for i in range(n_fold):
    model = GaussianNB()
    model.fit(train_x, train_y)

    test_pred = model.predict(test_x)
    from sklearn.metrics import accuracy_score
    temp_res = accuracy_score(test_y, test_pred)
    acc_temp+= temp_res
    
    precision, recall, f_value, support = precision_recall_fscore_support(test_y, test_pred, average='macro')
    
    prec_temp+=precision
    rec_temp += recall
    f1_temp += f_value
    
    print(f'Fold {i+1} Eval...   | ACC : {temp_res} | PREC : {precision} | REC : {recall} | F1-SCORE : {f_value} | SUPP : {support}')

res = acc_temp/n_fold
prec_res = prec_temp/n_fold
rec_res = rec_temp/n_fold
f1_res = f1_temp/n_fold

print(f'Average Accuracy Score({n_fold} Times Evaluation) : {res}')
print(f'Average Precision Score({n_fold} Times Evaluation) : {prec_res}')
print(f'Average Recall Score({n_fold} Times Evaluation) : {rec_res}')
print(f'Average F1 Score({n_fold} Times Evaluation) : {f1_res}')

Fold 1 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5703703703703704 | REC : 0.5844017094017094 | F1-SCORE : 0.5679012345679012 | SUPP : None
Fold 2 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5703703703703704 | REC : 0.5844017094017094 | F1-SCORE : 0.5679012345679012 | SUPP : None
Fold 3 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5703703703703704 | REC : 0.5844017094017094 | F1-SCORE : 0.5679012345679012 | SUPP : None
Fold 4 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5703703703703704 | REC : 0.5844017094017094 | F1-SCORE : 0.5679012345679012 | SUPP : None
Fold 5 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5703703703703704 | REC : 0.5844017094017094 | F1-SCORE : 0.5679012345679012 | SUPP : None
Fold 6 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5703703703703704 | REC : 0.5844017094017094 | F1-SCORE : 0.5679012345679012 | SUPP : None
Fold 7 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5703703703703704 | REC : 0.5844017094017094 | F1-SCORE : 0.5679012345679012

## SVM Classifier 

In [92]:
from sklearn.svm import SVC

acc_temp = 0
prec_temp = 0
rec_temp = 0
f1_temp = 0
n_fold = 10

for i in range(n_fold):
    model = SVC(kernel='linear')
    model.fit(train_x, train_y)

    test_pred = model.predict(test_x)
    from sklearn.metrics import accuracy_score
    temp_res = accuracy_score(test_y, test_pred)
    acc_temp+= temp_res
    
    precision, recall, f_value, support = precision_recall_fscore_support(test_y, test_pred, average='macro')
    
    prec_temp+=precision
    rec_temp += recall
    f1_temp += f_value
    
    print(f'Fold {i+1} Eval...   | ACC : {temp_res} | PREC : {precision} | REC : {recall} | F1-SCORE : {f_value} | SUPP : {support}')

res = acc_temp/n_fold
prec_res = prec_temp/n_fold
rec_res = rec_temp/n_fold
f1_res = f1_temp/n_fold

print(f'Average Accuracy Score({n_fold} Times Evaluation) : {res}')
print(f'Average Precision Score({n_fold} Times Evaluation) : {prec_res}')
print(f'Average Recall Score({n_fold} Times Evaluation) : {rec_res}')
print(f'Average F1 Score({n_fold} Times Evaluation) : {f1_res}')

Fold 1 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 2 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 3 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 4 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 5 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 6 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP : None
Fold 7 Eval...   | ACC : 0.5873015873015873 | PREC : 0.5520202020202021 | REC : 0.547008547008547 | F1-SCORE : 0.5382659547419274 | SUPP